In [ ]:
import os

In [ ]:
list_images = os.listdir(r'./datasets/train')

In [ ]:
len(list_images)   # total 25000 training images 4% we will take for validation set that is 1000

In [ ]:
cat_nos=0
dog_nos=0
switching ='cat'    #cat/dog
for i in list_images:
    val = i[:3]
    if val=='cat':
        cat_nos += 1
        if switching=='dog':
            switching='cat'
            print('switching at ',i,'with cat_nos ',cat_nos,' and dog_nos ',dog_nos)
    if val =='dog':
        dog_nos += 1
        if switching=='cat':
            switching='dog'
            print('switching at ',i,'with cat_nos ',cat_nos,' and dog_nos ',dog_nos)
    

In [ ]:
cat_nos, dog_nos   # equal no of dogs and cats

In [ ]:
cat_list = list_images[:12500]
dog_list = list_images[12500:]
len(cat_list), len(dog_list)

## Let's create folders to store them

In [ ]:
os.makedirs("./datasets/final/train/cats")
os.makedirs("./datasets/final/train/dogs")
os.makedirs("./datasets/final/test/cats")
os.makedirs("./datasets/final/test/dogs")

## Let's first create source folder

In [ ]:
os.makedirs("./datasets/source/cats")
os.makedirs("./datasets/source/dogs")

In [ ]:
## Let's first store split in source

In [ ]:
import shutil
from shutil import copyfile

In [ ]:
in_path = r"./datasets/train"
# store in cats
for img in cat_list:
    source_path = os.path.join(in_path,img)
    dest_path = os.path.join("./datasets/source/cats/",img)
    if os.path.getsize(source_path) >0:  # removes corrupt file
        copyfile(source_path, dest_path)

In [ ]:
for img in dog_list:
    source_path = os.path.join(in_path,img)
    dest_path = os.path.join("./datasets/source/dogs/",img)
    if os.path.getsize(source_path) >0:  # removes corrupt file
        copyfile(source_path, dest_path)

## send images to for split

In [ ]:
import random

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    images = os.listdir(SOURCE)
    images = random.sample(images,len(images))
    train_size = int(len(images)*SPLIT_SIZE)
    for img in images[:train_size]:
        PATH = os.path.join(SOURCE, img)
        if os.path.getsize(PATH) >0:
            DEST = os.path.join(TRAINING, img)
            copyfile(PATH, DEST)
    for img in images[train_size:]:
        PATH = os.path.join(SOURCE, img)
        if os.path.getsize(PATH)> 0:
            DEST = os.path.join(TESTING, img)
            copyfile(PATH, DEST)

# split cats
split_data('./datasets/source/cats', './datasets/final/train/cats','./datasets/final/test/cats', SPLIT_SIZE=.96)
# split dogs
split_data('./datasets/source/dogs', './datasets/final/train/dogs','./datasets/final/test/dogs', SPLIT_SIZE=.96)
            

## now data is splitted in train and test

In [ ]:
# first lets see how many of each we have
tr_c =r"./datasets/final/train/cats"
tr_d =r"./datasets/final/train/dogs"
te_c =r"./datasets/final/test/cats"
te_d =r"./datasets/final/test/dogs"

print("train cats:{} dogs:{} , test cats:{} dogs{}".format(len(os.listdir(tr_c)),len(os.listdir(tr_d)),len(os.listdir(te_c)),len(os.listdir(te_d))))

# RUN FROM Here >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

## use DataGen

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir=r"./datasets/final/train"
test_dir =r"./datasets/final/test"
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=.2,
                                  height_shift_range=.2,
                                  shear_range=.2,
                                  zoom_range=.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(299,299),
                                                   batch_size=64,
                                                   class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(test_dir,
                                                             target_size=(299,299),
                                                             batch_size=32,
                                                             class_mode ='binary'
                                                             )

In [ ]:
## Now create callbacks 
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", mode='min', patience=10)

In [ ]:
import tensorflow

class myCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("accuracy") is not None and logs.get("accuracy")>.9):
            print("\nReached 90% accuracy so stopping callback")
            self.model.stop_training=True

callbacks1 = myCallback()

## Now we will use transfer learning to create Inception Model

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense

In [ ]:


def my_InceptionV3_function(input_shape=(299, 299, 3), classes=2):
    
    base_model= tensorflow.keras.applications.InceptionV3(input_shape=input_shape,
                                                         include_top=False,
                                                         weights='imagenet')
    base_model.trainable = False
    
    # create the input layer (Same as the imageNetv2 input size)
    inputs = tensorflow.keras.Input(shape=input_shape) 
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(inputs) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 
    
    # Add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tensorflow.keras.layers.GlobalAveragePooling2D()(x) 
    #include dropout with probability of 0.2 to avoid overfitting
    x = tensorflow.keras.layers.Dropout(.2)(x)
    x = tensorflow.keras.layers.Dense(classes, activation='softmax')(x)
    InceptionV3_model = Model(inputs=inputs, outputs=x)
    # Let's compile it
    #LeNet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return InceptionV3_model

In [ ]:
model = my_InceptionV3_function()

In [ ]:
model.summary()

In [ ]:
#tensorflow.keras.utils.plot_model(model, to_file="model.png", show_shapes=False, show_layer_names=True, rankdir="TB", expand_nested=False, dpi=96, )

In [ ]:
# Compile our model
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model_history = model.fit(train_generator, steps_per_epoch=8, epochs=100, verbose=2, validation_data = validation_generator,validation_steps =8,callbacks=[early_stop, callbacks1] )

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.DataFrame(model_history.history).plot()

In [ ]:

a = pd.DataFrame(model_history.history)

In [ ]:
Final_DataFrame=pd.concat([Final_DataFrame, a])

In [ ]:
Final_DataFrame

In [ ]:
Final_DataFrame.plot()

In [ ]:
model.save('./saved_models/model1.h5')

## Let's load the model

In [ ]:
model = load_model('./saved_models/model1.h5')

In [ ]:
model.summary()

## predict

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir=r"./datasets/test/"
generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(299,299),
        batch_size=16,
        class_mode='categorical',  # only data, no labels
        shuffle=False)  # keep data in same order as labels

probabilities = model.predict(generator)

In [ ]:
probabilites

In [ ]:
import pandas as pd
Submission_df = pd.read_csv("./datasets/sampleSubmission.csv")

In [ ]:
Submission_df.head()

In [ ]:
type(probabilities)

In [ ]:
probabilities.shape

In [ ]:
Submission_df.info()

## check

In [ ]:
output = model.predict(validation_generator)

In [ ]:
output.shape

In [ ]:
import numpy as np
np.rint(output)

In [ ]:
pred=np.argmax(output, axis=1)

In [ ]:
y_true = validation_generator.classes
y_true

In [ ]:
pred.shape, y_true.shape

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
confusion_matrix(y_true, pred)

In [ ]:
print(classification_report(y_true, pred))

In [ ]:
final_output = np.argmax(probabilities, axis=1)

In [ ]:
len(final_output)

In [ ]:
Submission_df['label1']= final_output

In [ ]:
Submission_df.head()

In [ ]:
Submission_df['label'] = final_output

In [ ]:
Submission_df.drop('label1', axis=1,inplace=True)

In [ ]:
Submission_df.to_csv("./saved_outputs/sub1.csv")